<a href="https://colab.research.google.com/github/JonielOliveira/bertoti/blob/interacao-humano-computador/IHC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[1] – IMPLEMENTAÇÃO:**

## [1.1] – GOOGLE DRIVE:

### [1.1.1] – *GOOGLE DRIVE » CRIAR A CONEXÃO:*

Este código monta o Google Drive no ambiente do Google Colab, permitindo acessar e manipular arquivos armazenados no Drive diretamente do Colab. Cria os diretórios necessários, caso não existam.

In [ ]:
import os
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

# Caminhos dos diretórios principais
base_dir = "/content/drive/MyDrive/IHC"
data_dir = os.path.join(base_dir, "data")
log_dir = os.path.join(base_dir, "log")
credentials_dir = os.path.join(base_dir, "credentials")

# Subdiretórios dentro de "data"
data_subdirs = [
    "P1-input-audio",
    "P2-input-txt",
    "P3-llm-response",
    "P4-api-response",
    "P5-noticia",
    "P5-output-txt",
    "P6-output-audio"
]

# Função para criar diretórios se não existirem
def criar_diretorios(diretorios):
    for diretorio in diretorios:
        os.makedirs(diretorio, exist_ok=True)
        print(f"Diretório criado ou já existente: {diretorio}")

# Criar os diretórios principais
criar_diretorios([base_dir, data_dir, log_dir, credentials_dir])

# Criar os subdiretórios dentro de "data"
criar_diretorios([os.path.join(data_dir, subdir) for subdir in data_subdirs])


### [1.1.2] – *GOOGLE DRIVE » LER AS CHAVES DA APLICAÇÃO:*

Este código carrega um arquivo JSON contendo chaves de aplicação do Google Drive a partir do diretório montado no Colab.

In [ ]:
import json

# Caminho do arquivo JSON
arquivo_credencial = '/content/drive/MyDrive/IHC/credentials/credential_gcloud.json'

# Definir as credenciais em uma variável de ambiente
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = arquivo_credencial

# Abrindo o arquivo JSON para leitura
# with open(arquivo_chaves, 'r') as arquivo:
#    chaves = json.load(arquivo)

# Função para obter a chave associada ao nome
# def obter_chave(chaves, nome):
#  for chave in chaves:
#    if chave['name'] == nome:
#      return chave['key']
#  return None


## [1.2] – OPENAI WHISPER:

### [1.2.1] – *OPENAI WHISPER » INSTALAÇÃO DAS DEPENDÊNCIAS:*

Este código instala as dependências necessárias para usar o modelo OpenAI Whisper.

In [ ]:
!sudo apt-get install ffmpeg
!pip install torch numpy
!pip install openai-whisper

### [1.2.2] – *OPENAI WHISPER » IMPLEMENTAÇÃO DA FUNÇÃO:*

Este código define uma função que usa o modelo OpenAI Whisper para transcrever áudio em texto. A função carrega o modelo, transcreve o áudio em português e salva o texto resultante em um arquivo de saída.

In [ ]:
import warnings
import whisper

# Ignora warnings específicos
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def audio_para_texto(input_file, output_file):

  # Carrega o modelo Whisper
  model = whisper.load_model("base")

  # Transcreve o áudio, especificando o idioma como português
  result = model.transcribe(input_file, language="pt")

  # Salva o texto transcrito em um arquivo
  with open(output_file, "w") as f:
    f.write(result["text"])



## [1.4] – LARGE LANGUAGE MODEL (LLM):

### [1.4.1] – *LARGE LANGUAGE MODEL(LLM) » IMPLEMENTAÇÃO DA FUNÇÃO:*

Configura um modelo de linguagem (LLM) para gerar respostas a perguntas com base em exemplos fornecidos. Cria um pipeline de geração de texto usando o modelo Phi-3-mini-4k-instruct e define uma função que recebe uma pergunta, gera uma resposta identificando os tópicos de interesse, e retorna essa resposta.

In [ ]:
import transformers
import torch

# Define o modelo a ser utilizado
model_id = "microsoft/Phi-3-mini-4k-instruct"

# Cria o pipeline de geração de texto
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

# Função para gerar a resposta
def gerar_resposta(pergunta):

    # Orienta o LLM para que responda de uma forma previsível
    pergunta_exemplo_1 = "Bom dia, gostaria de saber como está o PIB."
    resposta_exemplo_1 = "PIB"
    pergunta_exemplo_2 = "Olá, poderia me informar sobre a agropecuária?"
    resposta_exemplo_2 = "agropecuária"
    pergunta_exemplo_3 = "Boa tarde, como está o IPCA ou INPC atualmente?"
    resposta_exemplo_3 = "IPCA, INPC"

    prompt = (f"Identifique quais os tópicos de interesse do usuário ao realizar uma pergunta e liste-os.\n\n"
              f"Você recebeu as seguintes perguntas e respostas como exemplos:\n\n"
              f"Pergunta exemplo 1: {pergunta_exemplo_1}\n"
              f"Resposta exemplo 1: {resposta_exemplo_1}\n\n"
              f"Pergunta exemplo 2: {pergunta_exemplo_2}\n"
              f"Resposta exemplo 2: {resposta_exemplo_2}\n\n"
              f"Pergunta exemplo 3: {pergunta_exemplo_3}\n"
              f"Resposta exemplo 3: {resposta_exemplo_3}\n\n"
              f"Agora, responda a nova pergunta com base no estilo da resposta exemplo.\n\n"
              f"Nova Pergunta: {pergunta}\n"
              f"Resposta:")

    resposta = pipeline(
        prompt,
        max_new_tokens=50,                  # Limita o número de tokens gerados
        do_sample=False,                    # Gera uma resposta determinística
        num_return_sequences=1,             # Retorna apenas uma sequência
        eos_token_id=50256,                 # Configura o token de fim de sequência (ajuda o modelo a saber quando parar)
        clean_up_tokenization_spaces=True,  # Remove espaços desnecessários
        stop_sequence="###"                 # Impede a repetição de prompt ou metadados
    )

    # Filtra a resposta apenas até o fim da primeira frase gerada

    # Obtém o texto gerado
    texto_gerado = resposta[0]['generated_text']

    # Encontrar o índice de "Resposta:"
    inicio_resposta = texto_gerado.find("Resposta:") + len("Resposta:")

    # Encontrar o índice de "\n\n" após "Resposta:"
    fim_resposta = texto_gerado.find("\n\n", inicio_resposta)

    # Extrair o intervalo entre "Resposta:" e "\n\n"
    if inicio_resposta != -1 and fim_resposta != -1:
        return texto_gerado[inicio_resposta:fim_resposta].strip()
    else:
        # Se não encontrar os delimitadores, retorna uma mensagem padrão ou ajusta conforme necessário
        return None



## [1.5] – API IBGE:

### [1.5.1] – *API IBGE » IMPLEMENTAÇÃO DA FUNÇÃO:*

Realiza consultas à API do IBGE (https://servicodados.ibge.gov.br/api/docs/noticias?versao=3) para buscar notícias relacionadas a um termo especificado. Define uma classe Noticia para armazenar e formatar as informações obtidas. A função realizar_consulta envia a consulta e a função obter_noticias extrai e retorna uma lista de notícias formatadas.

In [ ]:
import requests
from urllib.parse import quote

class Noticia:
    def __init__(self, titulo, introducao, data_publicacao, link):
        self.titulo = titulo
        self.introducao = introducao
        self.data_publicacao = data_publicacao
        self.link = link

    def to_dict(self):
        return {
            "titulo": self.titulo,
            "introducao": self.introducao,
            "data_publicacao": self.data_publicacao,
            "link": self.link
        }

    def __repr__(self):
        return f"Titulo={self.titulo!r}\nIntroducao={self.introducao!r}\nData_Publicacao={self.data_publicacao!r}\nLink={self.link!r}"

def realizar_consulta(solicitacao):
    url_base = 'https://servicodados.ibge.gov.br/api/v3/noticias/?busca='
    busca = quote(solicitacao)
    url = url_base + busca
    dados = requests.get(url).json()
    return dados

def obter_noticias(dados):
    if dados.get('count'):
        if dados['count'] > 0:
            noticias = []
            items = dados['items']
            if (len(items) > 0):
                for item in items:
                    nova_noticia = Noticia(
                        titulo=item.get('titulo'),
                        introducao=item.get('introducao'),
                        data_publicacao=item.get('data_publicacao'),
                        link=item.get('link')
                        )
                    noticias.append(nova_noticia)
                return noticias
            else:
                return None
        else:
            return None
    else:
        return None


## [1.6] – BOT TELEGRAM:

### [1.6.1] – *BOT TELEGRAM » INSTALAÇÃO DAS DEPENDÊNCIAS:*

 Instala bibliotecas necessárias para a criação de um bot no Telegram (python-telegram-bot), manipulação de modelos de linguagem (llama_cpp_python), e interação com a API do Telegram (pytelegrambotapi).

In [ ]:
!pip install python-telegram-bot==13.7
!pip install llama_cpp_python
!pip install pytelegrambotapi

### [1.6.2] – *BOT TELEGRAM » IMPLEMENTAÇÃO DA FUNÇÃO:*

Configura um bot do Telegram para receber mensagens de voz. O áudio é transcrito e interpretado para identificar tópicos de interesse. Com base nos tópicos, consulta a API do IBGE, gera um resumo das notícias relacionadas, converte o resumo em áudio, e envia esse áudio de volta ao usuário, junto com informações adicionais, como data e link da notícia.

In [ ]:
import json
from datetime import datetime
from google.colab import userdata

def encontrar_substring_em_listas(lista_strings_a, lista_strings_b):
    # Converte todas as strings em lista_strings_b para minúsculas
    lista_strings_b_lower = [s.lower() for s in lista_strings_b]

    # Itera sobre cada string em lista_strings_a
    for s_a in lista_strings_a:
        # Converte a string de lista_strings_a para minúsculas
        s_a_lower = s_a.lower()

        # Verifica se a string de lista_strings_a está contida em alguma das strings de lista_strings_b
        if any(s_a_lower in s_b_lower for s_b_lower in lista_strings_b_lower):
            return True

    return False

def agora_file():
  # Obtém a data e hora atual
  agora = datetime.now()

  # Formata a data e hora no formato desejado
  data_e_hora_formatada = agora.strftime("_%Y-%m-%d_%H-%M-%S")

  # Retorna a data e hora formatada
  return data_e_hora_formatada

def agora_log():
  # Obtém a data e hora atual
  agora = datetime.now()

  # Formata a data e hora no formato desejado
  data_e_hora_formatada = agora.strftime("[%d/%m/%Y %H:%M:%S]")

  # Retorna a data e hora formatada
  return data_e_hora_formatada

def hoje_log():
    # Obtém a data atual
    agora = datetime.now()

    # Formata a data no formato "ano-mes-dia"
    data_formatada = agora.strftime("%Y-%m-%d")

    # Retorna a data formatada
    return data_formatada

def create_log(indicador, file_location):
  status = ["Status 1: áudio de entrada recebido com sucesso! \n»»» (FILE)",
            "Status 2: transcrição da entrada gerada com sucesso! \n»»» (FILE)",
            "Status 3: resposta do LLM gerada com sucesso! \n»»» (FILE)",
            "Status 4: resposta do API IBGE obtida com sucesso! \n»»» (FILE)",
            "Status 5: notícia salva com sucesso! \n»»» (FILE)",
            "Status 6: áudio de saida gerado com sucesso! \n»»» (FILE)"]
  resposta = f"{agora_log()} {status[indicador - 1].replace("FILE", file_location)}"


import telebot

# Substitua 'TELEGRAM_API_KEY' pela sua chave de API do Telegram Bot
bot = telebot.TeleBot(userdata.get('TELEGRAM_API_KEY'))

@bot.message_handler(content_types=['voice'])
def handle_voice_message(message):

    locations = {
        "audio_entrada": "/content/drive/MyDrive/IHC/data/P1-input-audio/",
        "transcricao_entrada": "/content/drive/MyDrive/IHC/data/P2-input-txt/",
        "llm_response": "/content/drive/MyDrive/IHC/data/P3-llm-response/",
        "api_response": "/content/drive/MyDrive/IHC/data/P4-api-response/",
        "noticia": "/content/drive/MyDrive/IHC/data/P5-noticia/",
        "transcricao_saida": "/content/drive/MyDrive/IHC/data/P5-output-txt/",
        "audio_saida": "/content/drive/MyDrive/IHC/data/P6-output-audio/"
    }


    # Obtém o file_id do áudio de voz
    file_id = message.voice.file_id

    # Usa o file_id para obter informações sobre o arquivo
    file_info = bot.get_file(file_id)

    # Baixa o arquivo de áudio
    downloaded_file = bot.download_file(file_info.file_path)

    audio_entrada_file = f"{locations['audio_entrada']}audio_entrada{agora_file()}.ogg"

    # Salva o áudio localmente (Google Drive)
    with open(audio_entrada_file, 'wb') as new_file:
        new_file.write(downloaded_file)

    print(f"{agora_log()} Status 1: áudio de entrada recebido com sucesso! \n»»» ({audio_entrada_file})")

    transcricao_entrada_file = f"{locations['transcricao_entrada']}transcricao_entrada{agora_file()}.txt"

    audio_para_texto(audio_entrada_file, transcricao_entrada_file)

    # Abre o arquivo 'arquivo.txt' em modo de leitura
    with open(transcricao_entrada_file, 'r', encoding='utf-8') as arquivo:
        # Lê todo o conteúdo do arquivo e armazena na variável 'conteudo'
        conteudo = arquivo.read()

    print(f"{agora_log()} Status 2: transcrição da entrada gerada com sucesso! \n»»» ({transcricao_entrada_file})")

    ###------------------------------------------------------------------------------------------------------------
    ### STATUS 3:
    ###------------------------------------------------------------------------------------------------------------

    # Captura a resposta gerada
    generated_text = gerar_resposta(conteudo)

    llm_response_file = f"{locations['llm_response']}llm_response{agora_file()}.txt"

    # Salva o texto transcrito em um arquivo
    with open(llm_response_file, "w") as f:
      f.write(generated_text)

    print(f"{agora_log()} Status 3: resposta do LLM gerada com sucesso! \n»»» ({llm_response_file})")

    ###------------------------------------------------------------------------------------------------------------
    ### STATUS 4:
    ###------------------------------------------------------------------------------------------------------------

    dados_api = {}
    if generated_text is None:
      dados_api = {
                "count": 0,
                "page": 1,
                "totalPages": 1,
                "nextPage": 0,
                "previousPage": 0,
                "showingFrom": 0,
                "showingTo": 0,
                "items": []
              }
    else:
      lista_solicitacao = generated_text.split(",")
      lista_solicitacao = [item.strip() for item in lista_solicitacao]
      solicitacao = " ".join(lista_solicitacao)
      dados_api = realizar_consulta(solicitacao)

    api_response_file = f"{locations['api_response']}api_response{agora_file()}.json"

    # Salva o dicionário em um arquivo JSON
    with open(api_response_file, "w") as f:
      json.dump(dados_api, f, ensure_ascii=False, indent=4)

    print(f"{agora_log()} Status 4: resposta do API IBGE obtida com sucesso! \n»»» ({api_response_file})")

    ###------------------------------------------------------------------------------------------------------------
    ### STATUS 5:
    ###------------------------------------------------------------------------------------------------------------

    noticias = obter_noticias(dados_api)
    noticia = noticias[0].to_dict()

    for n in noticias:
      lista_textos = [n.titulo, n.introducao]
      if (encontrar_substring_em_listas(lista_solicitacao, lista_textos)):
        noticia = n.to_dict()
        break

    noticia_file = f"{locations['noticia']}noticia{agora_file()}.json"

    # Salva o dicionário em um arquivo JSON
    with open(noticia_file, "w") as f:
      json.dump(noticia, f, ensure_ascii=False, indent=4)

    print(f"{agora_log()} Status 5: notícia salva com sucesso! \n»»» ({noticia_file})")

    ###------------------------------------------------------------------------------------------------------------
    ### STATUS 6:
    ###------------------------------------------------------------------------------------------------------------

    audio_saida_file = f"{locations['audio_saida']}audio_saida{agora_file()}.ogg"

    texto_para_audio_json(noticia_file, audio_saida_file)

    print(f"{agora_log()} Status 6: áudio de saida gerado com sucesso! \n»»» ({audio_saida_file})")

    # Caminho para o arquivo de voz no diretório raiz
    voice = open(audio_saida_file, 'rb')

    # Enviar a mensagem de voz para o usuário
    bot.send_voice(message.chat.id, voice)

    reply = f"Data Publicação ({noticia.get('data_publicacao')})\nLink: {noticia.get('link')}"

    # Responde ao usuário
    bot.reply_to(message, reply)


# **[2] – TESTES:**

## [2.1] – GOOGLE DRIVE:

### [2.1.1] – *GOOGLE DRIVE » TESTE DE FUNÇÃO:*

In [ ]:
print(obter_chave(chaves, 'telegram'))
print(obter_chave(chaves, 'googlecloud'))

NameError: name 'obter_chave' is not defined

## [2.2] – OPENAI WHISPER:

### [2.2.1] – *OPENAI WHISPER » TESTE DA FUNÇÃO:*

In [ ]:
arquivo_entrada = "/content/drive/MyDrive/IHC/data/P1-input-audio/voice.ogg"
arquivo_saida = "/content/drive/MyDrive/IHC/data/P2-input-txt/transcricao2.txt"
audio_para_texto(arquivo_entrada, arquivo_saida)

NameError: name 'audio_para_texto' is not defined

## [2.3] – GOOGLE CLOUD:

### [2.3.1] – *GOOGLE CLOUD » TESTE DA FUNÇÃO:*

In [ ]:
arquivo_entrada = "/content/drive/MyDrive/IHC/data/P4-output-txt/perguntas_respostas.txt"
local_arquivo = "/content/drive/MyDrive/IHC/data/P5-output-audio/"
gerar_arquivo_de_audio(arquivo_entrada, local_arquivo)

NameError: name 'gerar_arquivo_de_audio' is not defined

## [2.4] – LARGE LANGUAGE MODEL (LLM):

### [2.4.1] – *LARGE LANGUAGE MODEL (LLM) » TESTE DA FUNÇÃO:*

In [ ]:
# Gera o texto a partir da entrada
response = pipeline("Hey how are you doing today?")

# Captura a resposta gerada
generated_text = response[0]["generated_text"]

# Exibe a resposta
print(generated_text)

## [2.5] – API IBGE:

### [2.5.1] – *API IBGE » TESTE DA FUNÇÃO:*

In [ ]:
solicitacao = input('> ')
resultado = realizar_consulta(solicitacao)
if resultado is None:
    print("Desculpe, nenhuma informação encontrada sobre o assunto!")
else:
    print(resultado)

# **[3] – EXECUÇÃO:**

Este trecho de código inicia a execução do bot do Telegram, permitindo que ele comece a monitorar e responder a mensagens recebidas em tempo real. O método bot.polling() mantém o bot em funcionamento contínuo, aguardando novas interações com os usuários.

In [ ]:
bot.polling()

# **[4] – FORA DE USO:**

## [1.3] – GOOGLE CLOUD TEXT-TO-SPEECH:


### [1.3.1] – *GOOGLE CLOUD TEXT-TO-SPEECH » INSTALAÇÃO DAS DEPENDÊNCIAS:*

Instala as bibliotecas necessárias para a utilização do Google Cloud Text-to-Speech (google-cloud-texttospeech), manipulação de áudio (pydub), e a ferramenta ffmpeg para converter e processar arquivos de áudio.

In [ ]:
!pip install google-cloud-texttospeech
!pip install pydub
!sudo apt-get install ffmpeg

### [1.3.2] – *GOOGLE CLOUD TEXT-TO-SPEECH » IMPLEMENTAÇÃO DA FUNÇÃO:*

Converte texto em áudio usando o Google Cloud Text-to-Speech. Primeiro, define as credenciais de acesso e cria uma função que gera um arquivo de áudio a partir de um texto e uma voz especificados. Outra função lê um arquivo JSON contendo um título e uma introdução, gera áudios separados para cada um com vozes diferentes, e os combina em um único arquivo .ogg.

In [ ]:
import os
from google.cloud import texttospeech
from google.cloud.texttospeech import SsmlVoiceGender
from pydub import AudioSegment

def gerar_audio_google_cloud(text, voice, output_file):

    client = texttospeech.TextToSpeechClient()

    # Configura a entrada como SSML
    synthesis_input = texttospeech.SynthesisInput(ssml=text)


    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)

    response = client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)

    with open(output_file, "wb") as out:
        out.write(response.audio_content)

# arquivo_entrada, arquivo_saida
def texto_para_audio_json(input_file, output_file):

  with open(input_file, 'r') as arquivo:
      noticia = json.load(arquivo)

  texto_titulo = noticia.get('titulo')
  ssml_titulo = f"""
                <speak>
                    <p>{texto_titulo}</p>
                </speak>
                """

  texto_introducao = noticia.get('introducao')
  ssml_introducao = f"""
                    <speak>
                        <p>{texto_introducao}</p>
                        <break time="1s"/>
                        <p>Saiba mais no link abaixo.</p>
                    </speak>
                    """

  # Escolha o gênero da voz: SsmlVoiceGender.MALE
  voz_homem = texttospeech.VoiceSelectionParams(
      language_code="pt-BR",
      name ="pt-BR-Standard-B",
      ssml_gender=texttospeech.SsmlVoiceGender.MALE
  )

  # Escolha o gênero da voz: SsmlVoiceGender.FEMALE
  voz_mulher = texttospeech.VoiceSelectionParams(
      language_code="pt-BR",
      name ="pt-BR-Standard-C",
      ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
  )

  # Encontra a posição do penúltimo '_'
  index = output_file.rfind('_', 0, output_file.rfind('_'))

  # Divide a string com base na posição do penúltimo '_'
  base = output_file[:index]
  suffix = output_file[index + 1:-4]

  # Monta as novas strings com as partes desejadas
  titulo_file = f"{base}_titulo_{suffix}.mp3"
  introducao_file = f"{base}_introducao_{suffix}.mp3"

  gerar_audio_google_cloud(ssml_titulo, voz_homem, titulo_file)
  gerar_audio_google_cloud(ssml_introducao, voz_mulher, introducao_file)

  audios = []
  audios.append(AudioSegment.from_mp3(titulo_file))
  audios.append(AudioSegment.from_mp3(introducao_file))

  audio = sum(audios)
  audio.export(output_file, format='ogg')
